In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path = './'

df_est = pd.read_csv(path + 'Input1_clientes_estructura.csv', sep=';')
df_venta = pd.read_csv(path + 'Input2_clientes_venta.csv', sep=';')

<h1>Exploracion</h1>

In [3]:
df_est.head()

,Cliente,Regional2,Gerencia2,SubCanal2,Categoria,Nevera
0,1,Regional 1,Gerencia_1,Subcanal_1,Categoria_1,0
1,2,Regional 1,Gerencia_1,Subcanal_1,Categoria_1,0
2,3,Regional 1,Gerencia_1,Subcanal_1,Categoria_1,0
3,4,Regional 1,Gerencia_1,Subcanal_1,Categoria_1,1
4,5,Regional 1,Gerencia_1,Subcanal_1,Categoria_2,1


In [4]:
df_venta.head()

,Año,Mes,Cliente,SegmentoPrecio2,Marca2,Cupo2,CapacidadEnvase2,Volumen,disc,nr
0,2019,5,10,SegmentoPrecio_1,Marca_1,Cupo_1,CapacidadEnvase_12,0.112229,-30.590603,900.328567
1,2019,5,10,SegmentoPrecio_1,Marca_2,Cupo_2,CapacidadEnvase_10,0.021734,0.000000,149.184463
2,2019,5,10,SegmentoPrecio_2,Marca_3,Cupo_2,CapacidadEnvase_10,0.043469,0.000000,359.625828
3,2019,5,10,SegmentoPrecio_1,Marca_1,Cupo_2,CapacidadEnvase_16,0.026345,-31.065261,134.748399
4,2019,5,10,SegmentoPrecio_1,Marca_4,Cupo_2,CapacidadEnvase_10,0.086938,0.000000,496.901005


In [5]:
df_venta.isnull().sum()

Año                 0
Mes                 0
Cliente             0
SegmentoPrecio2     0
Marca2              0
Cupo2               0
CapacidadEnvase2    0
Volumen             0
disc                0
nr                  0
dtype: int64

In [6]:
df_venta['Año'].unique()
df_venta.groupby('Año')['Mes'].unique()

Año
2019    [5, 6, 7, 9, 8, 10, 11, 12]
2020    [1, 2, 3, 5, 6, 7, 8, 9, 4]
Name: Mes, dtype: object

<h1>FEATURE ENGINEERING</h1>

In [7]:
# vamos a realizar una copia para dejar la data original tal cual esta
df_new_venta = df_venta.copy()
df_new_venta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406116 entries, 0 to 1406115
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Año               1406116 non-null  int64  
 1   Mes               1406116 non-null  int64  
 2   Cliente           1406116 non-null  int64  
 3   SegmentoPrecio2   1406116 non-null  object 
 4   Marca2            1406116 non-null  object 
 5   Cupo2             1406116 non-null  object 
 6   CapacidadEnvase2  1406116 non-null  object 
 7   Volumen           1406116 non-null  float64
 8   disc              1406116 non-null  float64
 9   nr                1406116 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 107.3+ MB


In [8]:
# vamos a unir las tres columnas de Marca2, Cupo2, CapacidadEnvase2 an una Sola
# de esta manera al realizar el filtro nos quedara mas facil
def createProduct(x):
    result = x['Marca2'] + '-' + x['Cupo2'] + '-' + x['CapacidadEnvase2']
    return result
df_new_venta['producto'] = df_new_venta.apply(createProduct, axis=1)
df_new_venta.drop(['Marca2', 'Cupo2', 'CapacidadEnvase2'], axis=1, inplace=True)
df_new_venta

,Año,Mes,Cliente,SegmentoPrecio2,Volumen,disc,nr,producto
0,2019,5,10,SegmentoPrecio_1,0.112229,-30.590603,900.328567,Marca_1-Cupo_1-CapacidadEnvase_12
1,2019,5,10,SegmentoPrecio_1,0.021734,0.000000,149.184463,Marca_2-Cupo_2-CapacidadEnvase_10
2,2019,5,10,SegmentoPrecio_2,0.043469,0.000000,359.625828,Marca_3-Cupo_2-CapacidadEnvase_10
3,2019,5,10,SegmentoPrecio_1,0.026345,-31.065261,134.748399,Marca_1-Cupo_2-CapacidadEnvase_16
4,2019,5,10,SegmentoPrecio_1,0.086938,0.000000,496.901005,Marca_4-Cupo_2-CapacidadEnvase_10
...,...,...,...,...,...,...,...,...
1406111,2020,8,20577,SegmentoPrecio_3,0.039122,0.000000,544.150314,Marca_39-Cupo_2-CapacidadEnvase_10
1406112,2020,7,20580,SegmentoPrecio_1,0.000000,0.000000,0.000000,Marca_38-Cupo_2-CapacidadEnvase_10
1406113,2020,7,20580,SegmentoPrecio_3,0.058683,0.000000,759.218996,Marca_39-Cupo_2-CapacidadEnvase_10
1406114,2020,9,20580,SegmentoPrecio_1,0.000000,0.000000,0.000000,Marca_38-Cupo_2-CapacidadEnvase_10


In [9]:
# de nuevo vamos a realizar una copia para mantener df_new_venta tal cual
# ya que nos puede servir para mas adelante
df_ventas_2 = df_new_venta.drop(['SegmentoPrecio2', 'Volumen', 'disc', 'nr'], axis=1)
df_ventas_2 = df_ventas_2.merge(df_est, on=['Cliente'])
df_ventas_2.head()

,Año,Mes,Cliente,producto,Regional2,Gerencia2,SubCanal2,Categoria,Nevera
0,2019,5,10,Marca_1-Cupo_1-CapacidadEnvase_12,Regional 1,Gerencia_3,Subcanal_1,Categoria_2,1
1,2019,5,10,Marca_2-Cupo_2-CapacidadEnvase_10,Regional 1,Gerencia_3,Subcanal_1,Categoria_2,1
2,2019,5,10,Marca_3-Cupo_2-CapacidadEnvase_10,Regional 1,Gerencia_3,Subcanal_1,Categoria_2,1
3,2019,5,10,Marca_1-Cupo_2-CapacidadEnvase_16,Regional 1,Gerencia_3,Subcanal_1,Categoria_2,1
4,2019,5,10,Marca_4-Cupo_2-CapacidadEnvase_10,Regional 1,Gerencia_3,Subcanal_1,Categoria_2,1


<h1>RandomForest</h1>


In [10]:
# Aqui vamos a realizar un nuevo feature Engineering donde cambiaremos a 0 y 1 si
# la marca a predecir fue comprada o no

def marca_uno(x):
  if str(x) == 'Marca_20-Cupo_3-CapacidadEnvase_9':
    return 1
  else:
    return 0

def marca_dos(x):
  if str(x) == 'Marca_16-Cupo_2-CapacidadEnvase_10':
    return 1
  else:
    return 0

def marca_tres(x):
  if str(x) == 'Marca_9-Cupo_3-CapacidadEnvase_12':
    return 1
  else:
    return 0

def marca_Inno_uno(x):
  if str(x) == 'Marca_38-Cupo_2-CapacidadEnvase_10':
    return 1
  else:
    return 0

def marca_Inno_dos(x):
  if str(x) == 'Marca_39-Cupo_2-CapacidadEnvase_10':
    return 1
  else:
    return 0

In [11]:
df_ventas_2['Marca1'] = df_ventas_2['producto'].apply(marca_uno)
df_ventas_2['Marca2'] = df_ventas_2['producto'].apply(marca_dos)
df_ventas_2['Marca3'] = df_ventas_2['producto'].apply(marca_tres)
df_ventas_2['MarcaInno1'] = df_ventas_2['producto'].apply(marca_Inno_uno)
df_ventas_2['MarcaInno2'] = df_ventas_2['producto'].apply(marca_Inno_dos)

In [12]:
aux = df_ventas_2[['Año', 'Mes', 'Cliente', 'Marca1', 'Marca2', 'Marca3', 'MarcaInno1', 'MarcaInno2']]
aux_group = aux.groupby(['Año', 'Mes', 'Cliente']).sum()
result = aux_group.reset_index()
result

,Año,Mes,Cliente,Marca1,Marca2,Marca3,MarcaInno1,MarcaInno2
0,2019,5,9,0,0,0,0,0
1,2019,5,10,0,0,0,0,0
2,2019,5,11,0,0,0,0,0
3,2019,5,12,0,1,0,0,0
4,2019,5,13,0,0,0,0,0
...,...,...,...,...,...,...,...,...
204631,2020,9,20411,0,0,0,1,1
204632,2020,9,20414,0,0,0,1,1
204633,2020,9,20434,0,0,0,1,1
204634,2020,9,20456,0,0,0,1,1


In [13]:
# este proceso se tiene que realizar para pasarlo por el RandomForest sin problemas
cliente_info = df_est.copy()

cliente_info['Regional2'] = cliente_info['Regional2'].astype('category')
cliente_info['Gerencia2'] = cliente_info['Gerencia2'].astype('category')
cliente_info['SubCanal2'] = cliente_info['SubCanal2'].astype('category')
cliente_info['Categoria'] = cliente_info['Categoria'].astype('category')

cliente_info['Regional2'] = cliente_info.Regional2.cat.codes
cliente_info['Gerencia2'] = cliente_info.Gerencia2.cat.codes
cliente_info['SubCanal2'] = cliente_info.SubCanal2.cat.codes
cliente_info['Categoria'] = cliente_info.Categoria.cat.codes

primer producto

In [14]:
primer_producto = result[['Año', 'Mes', 'Cliente', 'Marca1']]

In [15]:
primer_producto = primer_producto.merge(cliente_info, on='Cliente')
primer_producto

,Año,Mes,Cliente,Marca1,Regional2,Gerencia2,SubCanal2,Categoria,Nevera
0,2019,5,9,0,0,3,22,2,1
1,2019,6,9,0,0,3,22,2,1
2,2019,7,9,0,0,3,22,2,1
3,2019,8,9,0,0,3,22,2,1
4,2019,9,9,0,0,3,22,2,1
...,...,...,...,...,...,...,...,...,...
204631,2020,9,20186,0,0,10,29,0,0
204632,2020,9,20309,0,0,1,22,2,0
204633,2020,9,20312,0,0,7,27,1,0
204634,2020,9,20434,0,0,1,24,2,0


In [16]:
# primer Marca a predecir
y = primer_producto['Marca1']
x = primer_producto[['Año', 'Mes', 'Cliente', 'Regional2', 'Gerencia2', 'SubCanal2', 'Categoria', 'Nevera']]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [18]:
clf1 = RandomForestClassifier(max_depth=2, random_state=42)
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [19]:
answer = clf1.predict(X_test)
ans = clf1.predict_proba(X_test)
print(ans)
print(clf1.classes_)

[[0.992006   0.007994  ]
 [0.98616139 0.01383861]
 [0.97807069 0.02192931]
 ...
 [0.94371954 0.05628046]
 [0.98602662 0.01397338]
 [0.98382524 0.01617476]]
[0 1]


In [20]:
acc_score = accuracy_score(y_test, answer)
print(acc_score)
roc = roc_curve(y_test, answer)
print(roc)
auc_score = auc(roc[0], roc[1])
print(auc_score)

0.9736121970289289
(array([0., 1.]), array([0., 1.]), array([1, 0]))
0.5


segundo producto



In [21]:
segundo_producto = result[['Año', 'Mes', 'Cliente', 'Marca2']]
segundo_producto = segundo_producto.merge(cliente_info, on='Cliente')

# separacion de data y separacion entre train y test data
y = segundo_producto['Marca2']
x = segundo_producto[['Año', 'Mes', 'Cliente', 'Regional2', 'Gerencia2', 'SubCanal2', 'Categoria', 'Nevera']]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Modelo
clf2 = RandomForestClassifier(max_depth=2, random_state=42)
clf2.fit(X_train, y_train)

# Medidas
answer = clf2.predict(X_test)
acc_score = accuracy_score(y_test, answer)
print(acc_score)
roc = roc_curve(y_test, answer)
print(roc)
auc_score = auc(roc[0], roc[1])
print(auc_score)

0.9521354573885848
(array([0., 1.]), array([0., 1.]), array([1, 0]))
0.5


tercer producto

In [22]:
tercer_producto = result[['Año', 'Mes', 'Cliente', 'Marca3']]
tercer_producto = tercer_producto.merge(cliente_info, on='Cliente')

# separacion de data y separacion entre train y test data
y = tercer_producto['Marca3']
x = tercer_producto[['Año', 'Mes', 'Cliente', 'Regional2', 'Gerencia2', 'SubCanal2', 'Categoria', 'Nevera']]

# Modelo
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf3 = RandomForestClassifier(max_depth=2, random_state=42)
clf3.fit(X_train, y_train)

# Medidas
answer = clf3.predict(X_test)
acc_score = accuracy_score(y_test, answer)
print(acc_score)
roc = roc_curve(y_test, answer)
print(roc)
auc_score = auc(roc[0], roc[1])
print(auc_score)

0.7238809616888194
(array([0., 1.]), array([0., 1.]), array([1, 0]))
0.5


cuarto producto

In [23]:
cuarto_producto = result[['Año', 'Mes', 'Cliente', 'MarcaInno1']]
cuarto_producto = cuarto_producto.merge(cliente_info, on='Cliente')

# separacion de data y separacion entre train y test data
y = cuarto_producto['MarcaInno1']
x = cuarto_producto[['Año', 'Mes', 'Cliente', 'Regional2', 'Gerencia2', 'SubCanal2', 'Categoria', 'Nevera']]

# Modelo
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf4 = RandomForestClassifier(max_depth=2, random_state=42)
clf4.fit(X_train, y_train)

# Medidas
answer = clf4.predict(X_test)
acc_score = accuracy_score(y_test, answer)
print(acc_score)
roc = roc_curve(y_test, answer)
print(roc)
auc_score = auc(roc[0], roc[1])
print(auc_score)

0.9568999218139171
(array([0., 1.]), array([0., 1.]), array([1, 0]))
0.5


quinto producto

In [24]:
quinto_producto = result[['Año', 'Mes', 'Cliente', 'MarcaInno2']]
quinto_producto = quinto_producto.merge(cliente_info, on='Cliente')

# separacion de data y separacion entre train y test data
y = quinto_producto['MarcaInno2']
x = quinto_producto[['Año', 'Mes', 'Cliente', 'Regional2', 'Gerencia2', 'SubCanal2', 'Categoria', 'Nevera']]

# Modelo
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf5 = RandomForestClassifier(max_depth=2, random_state=42)
clf5.fit(X_train, y_train)

answer = clf5.predict(X_test)
acc_score = accuracy_score(y_test, answer)
print(acc_score)
roc = roc_curve(y_test, answer)
print(roc)
auc_score = auc(roc[0], roc[1])
print(auc_score)

0.9568999218139171
(array([0., 1.]), array([0., 1.]), array([1, 0]))
0.5


LLENANDO TEST

In [25]:
df_test = pd.read_csv(path + 'Input3_clientes_test.csv', sep=';')

In [26]:
df_test

,Cliente,Marca1,Marca2,Marca3,Marca_Inno1,Marca_Inno2
0,10,NaN,NaN,NaN,NaN,NaN
1,12,NaN,NaN,NaN,NaN,NaN
2,14,NaN,NaN,NaN,NaN,NaN
3,15,NaN,NaN,NaN,NaN,NaN
4,18,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
9297,20186,NaN,NaN,NaN,NaN,NaN
9298,20261,NaN,NaN,NaN,NaN,NaN
9299,20309,NaN,NaN,NaN,NaN,NaN
9300,20360,NaN,NaN,NaN,NaN,NaN


In [27]:
test = df_test[['Cliente']]
test = test.merge(df_est, on='Cliente')
test['Año'] = 2020
test['Mes'] = 10
test = test[['Año', 'Mes', 'Cliente', 'Regional2', 'Gerencia2', 'SubCanal2', 'Categoria', 'Nevera']]
test

,Año,Mes,Cliente,Regional2,Gerencia2,SubCanal2,Categoria,Nevera
0,2020,10,10,Regional 1,Gerencia_3,Subcanal_1,Categoria_2,1
1,2020,10,12,Regional 1,Gerencia_3,Subcanal_4,Categoria_4,1
2,2020,10,14,Regional 1,Gerencia_3,Subcanal_1,Categoria_2,1
3,2020,10,15,Regional 1,Gerencia_2,Subcanal_3,Categoria_2,1
4,2020,10,18,Regional 1,Gerencia_3,Subcanal_1,Categoria_5,1
...,...,...,...,...,...,...,...,...
9297,2020,10,20186,Regional 1,Gerencia_9,Subcanal_7,Categoria_1,0
9298,2020,10,20261,Regional 1,Gerencia_10,Subcanal_1,Categoria_2,0
9299,2020,10,20309,Regional 1,Gerencia_10,Subcanal_3,Categoria_3,0
9300,2020,10,20360,Regional 1,Gerencia_10,Subcanal_8,Categoria_1,0


In [28]:
test['Regional2'] = test['Regional2'].astype('category')
test['Gerencia2'] = test['Gerencia2'].astype('category')
test['SubCanal2'] = test['SubCanal2'].astype('category')
test['Categoria'] = test['Categoria'].astype('category')

test['Regional2'] = test.Regional2.cat.codes
test['Gerencia2'] = test.Gerencia2.cat.codes
test['SubCanal2'] = test.SubCanal2.cat.codes
test['Categoria'] = test.Categoria.cat.codes

In [29]:
prob1 = clf1.predict_proba(test)[:, 1]
prob2 = clf2.predict_proba(test)[:, 1]
prob3 = clf3.predict_proba(test)[:, 1]
prob4 = clf4.predict_proba(test)[:, 1]
prob5 = clf5.predict_proba(test)[:, 1]

In [30]:
test['Marca1'] = prob1
test['Marca2'] = prob2
test['Marca3'] = prob3
test['Marca_Inno1'] = prob4
test['Marca_Inno2'] = prob5

In [31]:
final = test[['Cliente', 'Marca1', 'Marca2', 'Marca3', 'Marca_Inno1', 'Marca_Inno2']]
final

,Cliente,Marca1,Marca2,Marca3,Marca_Inno1,Marca_Inno2
0,10,0.055393,0.050081,0.256060,0.084214,0.084214
1,12,0.053019,0.080051,0.302099,0.089676,0.089676
2,14,0.055393,0.050081,0.256060,0.084214,0.084214
3,15,0.052316,0.048440,0.251948,0.084214,0.084214
4,18,0.056569,0.081615,0.306210,0.089874,0.089874
...,...,...,...,...,...,...
9297,20186,0.084881,0.025497,0.211585,0.216928,0.216928
9298,20261,0.090971,0.032706,0.220766,0.210166,0.210166
9299,20309,0.089158,0.031441,0.263293,0.220889,0.220889
9300,20360,0.088400,0.026031,0.216274,0.210332,0.210332


In [32]:
df_test.columns

Index(['Cliente', 'Marca1', 'Marca2', 'Marca3', 'Marca_Inno1', 'Marca_Inno2'], dtype='object')

In [33]:
final.columns = df_test.columns
final

,Cliente,Marca1,Marca2,Marca3,Marca_Inno1,Marca_Inno2
0,10,0.055393,0.050081,0.256060,0.084214,0.084214
1,12,0.053019,0.080051,0.302099,0.089676,0.089676
2,14,0.055393,0.050081,0.256060,0.084214,0.084214
3,15,0.052316,0.048440,0.251948,0.084214,0.084214
4,18,0.056569,0.081615,0.306210,0.089874,0.089874
...,...,...,...,...,...,...
9297,20186,0.084881,0.025497,0.211585,0.216928,0.216928
9298,20261,0.090971,0.032706,0.220766,0.210166,0.210166
9299,20309,0.089158,0.031441,0.263293,0.220889,0.220889
9300,20360,0.088400,0.026031,0.216274,0.210332,0.210332


In [34]:
final.columns

Index(['Cliente', 'Marca1', 'Marca2', 'Marca3', 'Marca_Inno1', 'Marca_Inno2'], dtype='object')

In [35]:
final.to_csv('input4_clientes_test.csv', sep=',', encoding='utf-8', index=False)

In [36]:
prueba = pd.read_csv(path + 'input4_clientes_test.csv', sep=',')
prueba

,Cliente,Marca1,Marca2,Marca3,Marca_Inno1,Marca_Inno2
0,10,0.055393,0.050081,0.256060,0.084214,0.084214
1,12,0.053019,0.080051,0.302099,0.089676,0.089676
2,14,0.055393,0.050081,0.256060,0.084214,0.084214
3,15,0.052316,0.048440,0.251948,0.084214,0.084214
4,18,0.056569,0.081615,0.306210,0.089874,0.089874
...,...,...,...,...,...,...
9297,20186,0.084881,0.025497,0.211585,0.216928,0.216928
9298,20261,0.090971,0.032706,0.220766,0.210166,0.210166
9299,20309,0.089158,0.031441,0.263293,0.220889,0.220889
9300,20360,0.088400,0.026031,0.216274,0.210332,0.210332
